<a href="https://colab.research.google.com/github/binhnguyenManagingLife/PainGuide/blob/main/PainGuideToRealPain_Legacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview
- Convert excel raw content of Pain Guide into Markdowns
- Extracting

```
Primary Title
Secondary Title
Content
Deep URL
```

Assumptions:
- Each file has to have headers with English in it
- Each file needs to have Total number of articles in Overview sheet

Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import matplotlib
import random
import copy

Connect GG drive

In [ ]:
import gspread
from google.colab import auth
from google.auth import default

# Connect GG drive
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

# Authenticate user
auth.authenticate_user()

# Get credentials and initialize gspread client
creds, _ = default()
gc = gspread.authorize(creds)

# !pip install gspread pandas gspread_dataframe
# !pip install --upgrade gspread google-auth google-auth-oauthlib pandas

Mounted at /content/gdrive/


# Load Datasets

In [ ]:
# Function to read all of the sheet files in the spreadsheet

def read_sheets (spreadsheet):

  # Get all sheet names
  sheet_names = [sheet.title for sheet in spreadsheet.worksheets()]

  # Initialize a dictionary to store DataFrames for each sheet
  sheets_data = {}

  # Loop over each sheet, get the data, and store it in the dictionary
  for sheet_name in sheet_names:
      sheet = spreadsheet.worksheet(sheet_name)
      data = sheet.get_all_records()  # Get all rows as a list of dictionaries
      df = pd.DataFrame(data)         # Convert to pandas DataFrame
      sheets_data[sheet_name] = df    # Store the DataFrame with sheet name as key

  # # Display the DataFrame for each sheet
  # for sheet_name, df in sheets_data.items():
  #     print(f"Sheet: {sheet_name}")
  #     print(df)

  return (sheets_data,sheet_names)

In [ ]:
# Import respective data

# file1 = '/content/gdrive/MyDrive/LLM /Knowledge Base Material/Pain Guide/Pain Guide Articles - Dec 22.gsheet'
# file2 = '/content/gdrive/MyDrive/LLM /Knowledge Base Material/Pain Guide/Pain Guide Articles - Feb 26.gsheet'
# file3 = '/content/gdrive/MyDrive/LLM /Knowledge Base Material/Pain Guide/Pain Guide Articles - Jun 3.gsheet'

spreadsheet = gc.open("Pain Guide Articles - Dec 22")
sheets_data, sheet_names = read_sheets (spreadsheet)


In [ ]:
# Find the number of articles per file
df = sheets_data [sheet_names[0]]

# Loop through rows in column 0 to find the number of articles per file
for row in range (df.shape[0]):
  col = 0
  if 'Total articles:' in str(df.iloc[row,col]):
     break

# Find index of actual number
n_articles = df.iloc[row, col+1]
print ("Total numner of articles in this excel sheet is %d" %n_articles)

# # Uncomment below to visualize
# sheets_data[sheet_names[0]]

Total numner of articles in this excel sheet is 44


# Extract and put into MarkDown Format

In [ ]:
df = sheets_data [sheet_names[12]]
prim_title = df['English'][0]
sec_title = df['English'][1]
content = df['English'][2]

defn_loc = 3 # This location is always at index 3
defn = df['English'][defn_loc:]
defn_len = len (defn[defn.str.strip() != ''])

In [ ]:
article_text = (f"# Primary title: {prim_title}\n\
## Secondary title: {sec_title}\n\
### Content: {content}\n\
### Defintions\n")

print (article_text)

for i in range (defn_loc,defn_loc+defn_len):
  print (f"{defn[i]}\n\n")

# Primary title: Find Your Optimal Activity Level
## Secondary title: Improve your tolerance for regular physical activity to better cope with pain
### Content: Consult with your doctor about the type of activity that you can do safely and that will be beneficial for you. Your activity regiment should be tailored to your specific pain condition.

Keeping active is a helpful coping strategy for chronic pain. But when you live with pain, it can be hard to balance how much physical activity you do on a regular basis.
 
On a good day, you may push too hard to get things done and cause a flare-up of pain. Then you don’t do any activity as you wait for the pain to go down. This pattern can feel like a roller coaster that derails your plans for staying active and keeping pain at bay. It’s called the “boom or bust” cycle.

How can you find your optimal activity level?
 
Your tolerance for physical activity can be reduced by pain from too much or too little physical activity. To expand your tol

In [ ]:
with open('/content/gdrive/MyDrive/Google Collab/Example.md', 'w') as writefile:
    writefile.write(article_text)

for i in range (defn_loc,defn_loc+defn_len):
  with open('/content/gdrive/MyDrive/Google Collab/Example.md', 'a') as writefile:
    writefile.write(f"{defn[i]}\n\n")


# Final Loop it through all of the sheets

In [ ]:
################################### Creating the file
filename = 'PainGuideArticles1'
path = '/content/gdrive/MyDrive/Google Collab/'+filename+'.md'
with open(path, 'w') as writefile:
  writefile.write('')

################################### Load Data

spreadsheet = gc.open("Pain Guide Articles - Dec 22")
sheets_data, sheet_names = read_sheets (spreadsheet)

# Find the number of articles per file
df = sheets_data [sheet_names[0]]
print ("File loaded successfully")

# Loop through rows in column 0 to find the number of articles per file
for row in range (df.shape[0]):
  col = 0
  if 'Total articles:' in str(df.iloc[row,col]):
     break

# Find index of actual number
n_articles = df.iloc[row, col+1]
print ("Total numner of articles in this excel sheet is %d" %n_articles)

################################### Loop through sheets to export into MD

for i in range (1,n_articles+1):

  # Loop through all of the sheets
  df = sheets_data [sheet_names[i]]
  print(f"Analyzing Sheet {sheet_names[i]}")

  # Extract the semantic headings
  prim_title = df['English'][0]
  sec_title = df['English'][1]
  content = df['English'][2]
  defn_loc = 3 # This location is always at index 3
  defn = df['English'][defn_loc:]
  defn_len = len (defn[defn.str.strip() != ''])

  # Preparing the right formatting and semantics of the markdown
  article_text = (f"# Primary title: {prim_title}\n\
  ## Secondary title: {sec_title}\n\
  ### Content: {content}\n\
  ### Defintions\n")

  with open(path, 'a') as writefile:
    writefile.write(article_text)

  for i in range (defn_loc,defn_loc+defn_len):
    with open(path, 'a') as writefile:
      writefile.write(f"{defn[i]}\n\n")
  print ("Written sucessfully")


File loaded successfully
Total numner of articles in this excel sheet is 44
Analyzing Sheet ✅ Is Your Pain Acute or Chronic?
Written sucessfully
Analyzing Sheet ✅ Acute vs. Chronic Pain
Written sucessfully
Analyzing Sheet ✅ Pain Changes Your Nervous System
Written sucessfully
Analyzing Sheet ✅ Why Does Pain Have Different Characteristics?
Written sucessfully
Analyzing Sheet ✅ Characteristics as Clues to Your Pain
Written sucessfully
Analyzing Sheet ✅ Putting Pain into Words
Written sucessfully
Analyzing Sheet ✅ Learn About Dull Pain
Written sucessfully
Analyzing Sheet ✅ Understand Throbbing Pain
Written sucessfully
Analyzing Sheet ✅ Learn About Sharp Pain
Written sucessfully
Analyzing Sheet ✅ The Benefits of Gentle Movement
Written sucessfully
Analyzing Sheet ✅ How to Exercise Safely
Written sucessfully
Analyzing Sheet ✅ Find Your Optimal Activity Level
Written sucessfully
Analyzing Sheet ✅ Is Coping a Bad Word?
Written sucessfully
Analyzing Sheet ✅ Different Ways of Coping
Written suc

In [ ]:
################################### Creating the file
filename = 'PainGuideArticles2'
path = '/content/gdrive/MyDrive/Google Collab/'+filename+'.md'
with open(path, 'w') as writefile:
  writefile.write('')

################################### Load Data

spreadsheet = gc.open("Pain Guide Articles - Feb 26")
sheets_data, sheet_names = read_sheets (spreadsheet)

# Find the number of articles per file
df = sheets_data [sheet_names[0]]
print ("File loaded successfully")

# Loop through rows in column 0 to find the number of articles per file
for row in range (df.shape[0]):
  col = 0
  if 'Total articles:' in str(df.iloc[row,col]):
     break

# Find index of actual number
n_articles = df.iloc[row, col+1]
print ("Total numner of articles in this excel sheet is %d" %n_articles)

################################### Loop through sheets to export into MD

for i in range (1,n_articles+1):

  # Loop through all of the sheets
  df = sheets_data [sheet_names[i]]
  print(f"Analyzing Sheet {sheet_names[i]}")

  # Extract the semantic headings
  prim_title = df['English'][0]
  sec_title = df['English'][1]
  content = df['English'][2]
  defn_loc = 3 # This location is always at index 3
  defn = df['English'][defn_loc:]
  defn_len = len (defn[defn.str.strip() != ''])

  # Preparing the right formatting and semantics of the markdown
  article_text = (f"# Primary title: {prim_title}\n\
  ## Secondary title: {sec_title}\n\
  ### Content: {content}\n\
  ### Defintions\n")

  with open(path, 'a') as writefile:
    writefile.write(article_text)

  for i in range (defn_loc,defn_loc+defn_len):
    with open(path, 'a') as writefile:
      writefile.write(f"{defn[i]}\n\n")
  print ("Written sucessfully")


File loaded successfully
Total numner of articles in this excel sheet is 8
Analyzing Sheet Pain After Surgery: What to Expect
Written sucessfully
Analyzing Sheet Preparing to Manage Pain After Surgery
Written sucessfully
Analyzing Sheet Why Pain Can Persist After Surgery
Written sucessfully
Analyzing Sheet How to Reduce your Risk of Ongoing Pain After Surgery
Written sucessfully
Analyzing Sheet How do you know you’re ready for more?
Written sucessfully
Analyzing Sheet Sleep and Pain
Written sucessfully
Analyzing Sheet Sleep Health-Self Assessment
Written sucessfully
Analyzing Sheet Goal-Setting to Improve How You Feel
Written sucessfully


In [ ]:
################################### Creating the file
filename = 'PainGuideArticles3'
path = '/content/gdrive/MyDrive/Google Collab/'+filename+'.md'
with open(path, 'w') as writefile:
  writefile.write('')

################################### Load Data

spreadsheet = gc.open("Pain Guide Articles - Jun 3")
sheets_data, sheet_names = read_sheets (spreadsheet)

# Find the number of articles per file
df = sheets_data [sheet_names[0]]
print ("File loaded successfully")

# Loop through rows in column 0 to find the number of articles per file
for row in range (df.shape[0]):
  col = 0
  if 'Total articles:' in str(df.iloc[row,col]):
     break

# Find index of actual number
n_articles = df.iloc[row, col+1]
print ("Total numner of articles in this excel sheet is %d" %n_articles)

################################### Loop through sheets to export into MD

for i in range (1,n_articles+1):

  # Loop through all of the sheets
  df = sheets_data [sheet_names[i]]
  print(f"Analyzing Sheet {sheet_names[i]}")

  # Extract the semantic headings
  prim_title = df['English'][0]
  sec_title = df['English'][1]
  content = df['English'][2]
  defn_loc = 3 # This location is always at index 3
  defn = df['English'][defn_loc:]
  defn_len = len (defn[defn.str.strip() != ''])

  # Preparing the right formatting and semantics of the markdown
  article_text = (f"# Primary title: {prim_title}\n\
  ## Secondary title: {sec_title}\n\
  ### Content: {content}\n\
  ### Defintions\n")

  with open(path, 'a') as writefile:
    writefile.write(article_text)

  for i in range (defn_loc,defn_loc+defn_len):
    with open(path, 'a') as writefile:
      writefile.write(f"{defn[i]}\n\n")
  print ("Written sucessfully")


File loaded successfully
Total numner of articles in this excel sheet is 3
Analyzing Sheet Managing Anxiety Before Surgery
Written sucessfully
Analyzing Sheet Understanding Anxiety Before Surgery
Written sucessfully
Analyzing Sheet Preparing Ahead for Surgery
Written sucessfully
